# Содержание
- [K-means](#1)
- [DBSCAN](#2)
- [Агломеративная кластеризация](#3)

<a name='1'></a>
## K-Means

K-Means (k-средних) — это алгоритм кластеризации, который используется для разделения набора данных на группы (кластеры) на основе их схожести. Алгоритм старается минимизировать суммарное квадратичное отклонение между точками внутри каждого кластера и центроидами (средними значениями) этих точек.

Процесс работы алгоритма k-Means следующий:

1. Инициализация: Выбирается количество кластеров k и случайно инициализируются k центроидов.
2. Присваивание точек к кластерам: Каждая точка данных присваивается к ближайшему центроиду на основе евклидова расстояния или другой метрики.
3. Пересчет центроидов: Вычисляются новые центроиды путем нахождения средних значений точек в каждом кластере.
4. Повторение шагов 2 и 3: Шаги 2 и 3 повторяются до сходимости алгоритма, то есть до тех пор, пока точки перестают изменять свою принадлежность к кластерам или достигнут предел максимального количества итераций.

В результате работы алгоритма k-Means мы получаем k кластеров, в каждом из которых точки схожи между собой и отличаются от точек в других кластерах. Кластеризация данных с помощью k-Means может быть полезна для группировки и классификации данных, выявления скрытых закономерностей и понимания структуры данных.

In [ ]:
import numpy as np

In [ ]:
def kmeans(X, K, max_iters=100):
    # Инициализация центроидов случайным образом
    centroids = X[np.random.choice(range(len(X)), K, replace=False)]

    for _ in range(max_iters):
        # Нахождение ближайшего центроида для каждой точки
        labels = np.argmin(np.linalg.norm(X[:, np.newaxis] - centroids, axis=-1), axis=-1)

        # Обновление центроидов
        new_centroids = np.array([X[labels == k].mean(axis=0) for k in range(K)])

        # Проверка условия сходимости
        if np.all(centroids == new_centroids):
            break

        centroids = new_centroids

    return centroids, labels

In [ ]:
# Пример использования
X = np.array([[1, 2], [1.5, 1.8], [5, 8], [8, 8], [1, 0.6], [9, 11]])
K = 2

centroids, labels = kmeans(X, K)
print("Центроиды:")
print(centroids)
print("Метки кластеров:")
print(labels)

### Недостатки алгоритма

K-Means не справляется с кластерами сложной формы (например, кольцевыми или пересекающимися кластерами). Метод также зависит от начальной инициализации центроидов, что может привести к различным результатам.
Требуется заранее задавать количество кластеров k.

### Методы выбора k

- Метод локтя (Elbow Method): Построить график зависимости суммы внутрикластерных квадратов расстояний (`inertia`) от количества кластеров и выбрать k, где график «ломается». `inertia` — это метрика, которая показывает, насколько близко точки внутри одного кластера расположены к своему центроиду. Чем меньше значение, тем плотнее точки внутри кластера.

- Силуэтный анализ (Silhouette Score): Оценить качество кластеризации на основе расстояния между точками и их кластерами.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

# Генерация искусственных данных
X, y = make_blobs(n_samples=300, centers=5, cluster_std=1.0, random_state=42)

# Список для хранения значения "inertia" (суммы внутрикластерных квадратов расстояний)
inertia = []

# Диапазон количества кластеров для тестирования
k_values = range(1, 11)

# Построение модели K-Means для каждого значения k
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)

# Построение графика метода локтя
plt.figure(figsize=(8, 5))
plt.plot(k_values, inertia, marker='o', linestyle='--')
plt.xlabel('Количество кластеров (k)')
plt.ylabel('Inertia (внутрикластерная сумма квадратов)')
plt.title('Метод локтя для выбора оптимального k')
plt.xticks(k_values)
plt.grid(True)
plt.show()


Оптимальное значение K соответствует точке, где `inertia` перестает значительно уменьшаться. В данном случае, имеет смысл зафиксировать K=4 (занятно, что данные при этом сгенерированы исходя из K=5).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

# Генерация искусственных данных
X, y = make_blobs(n_samples=300, centers=5, cluster_std=1.0, random_state=42)

# Значения k для визуализации
k_values = [2, 3, 4, 10]

# Создание графиков
fig, axes = plt.subplots(1, len(k_values), figsize=(20, 5))

for ax, k in zip(axes, k_values):
    # Инициализация и обучение K-Means
    kmeans = KMeans(n_clusters=k, random_state=42)
    y_kmeans = kmeans.fit_predict(X)

    # Визуализация кластеров
    ax.scatter(X[:, 0], X[:, 1], c=y_kmeans, cmap='viridis', s=50, alpha=0.7)
    ax.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1],
               c='red', marker='x', s=100, label='Центроиды')
    ax.set_title(f'K-Means с K={k}')
    ax.set_xlabel('Признак 1')
    ax.set_ylabel('Признак 2')
    ax.legend()

plt.tight_layout()
plt.show()


<a name='2'></a>
## DBSCAN

DBSCAN (Density-Based Spatial Clustering of Applications with Noise) - это алгоритм кластеризации данных, основанный на плотности. Он идентифицирует кластеры, исходя из плотности точек в пространстве данных, а также обнаруживает выбросы, которые не принадлежат ни к одному кластеру.

Процесс работы алгоритма DBSCAN следующий:

1. Выбор начальной точки: Выбирается случайная точка, которая еще не была посещена и не является выбросом.
2. Поиск плотной области: Алгоритм расширяется от выбранной начальной точки, посещая ближайшие точки в заданном радиусе epsilon. Если в заданной окрестности (эпсилон-окрестности) находится минимальное количество точек, необходимое для формирования кластера, то точка считается ядром (core point).
3. Расширение кластера: Для каждого ядра образуется кластер, который включает все точки, достижимые из этого ядра в заданном радиусе epsilon. Для этого рекурсивно ищутся все плотные точки в окрестности ядра.
4. Поиск выбросов: Точки, которые не достижимы из ядерных точек, считаются выбросами (noise points). Они не принадлежат ни к одному кластеру.
5. Повторение шагов 2-4: Шаги 2-4 повторяются для всех непосещенных точек данных до тех пор, пока все точки не будут просмотрены.

### Основные параметры
- ε (eps): Радиус окрестности точки.
- MinPts: Минимальное количество точек для формирования плотного кластера.

### Преимущества DBSCAN
- Работает с кластерами произвольной формы.
- Способен выявлять шум (выбросы).


### Недостатки DBSCAN
- Чувствителен к параметрам ε и MinPts.
- Плохо масштабируется для больших данных.

In [ ]:
class DBSCAN:
    def __init__(self, eps, min_samples):
        self.eps = eps
        self.min_samples = min_samples
        self.labels = None

    def euclidean_distance(self, x1, x2):
        return np.sqrt(np.sum((x1 - x2) ** 2))

    def fit(self, X):
        self.labels = np.zeros(len(X))  # Инициализируем массив меток кластера нулями для каждой точки в датасете
        cluster_label = 0  # Инициализируем счетчик меток кластера

        for i in range(len(X)):  # Проходим по всем точкам в датасете
            if self.labels[i] != 0:  # Если точка уже имеет метку кластера, переходим к следующей точке
                continue

            neighbors = self._find_neighbors(X, i)  # Находим соседей для текущей точки

            if len(neighbors) < self.min_samples:  # Если количество соседей меньше минимального требования
                self.labels[i] = -1  # Присваиваем точке метку выброса (-1)
            else:
                cluster_label += 1  # Инкрементируем счетчик меток кластера
                self._expand_cluster(X, i, neighbors, cluster_label)  # Расширяем кластер, начиная с текущей точки


    def _expand_cluster(self, X, point_index, neighbors, cluster_label):
        """
        В этом методе сначала присваивается выбранной точке `point_index` метка кластера `cluster_label`.

        Затем идет цикл `while`, который выполняется, пока не исчерпаны все соседи в списке `neighbors`.

        На каждой итерации цикла берется очередной сосед из списка `neighbors` и проверяется его метка кластера:

        - Если метка равна `-1`, это означает, что сосед не имеет метки кластера.
            В этом случае ему присваивается метка кластера `cluster_label`.
        - Если метка равна `0`, это означает, что сосед не принадлежит ни одному кластеру.
            В этом случае ему также присваивается метка кластера `cluster_label`.
        Затем для этого соседа находятся новые соседи с помощью метода `_find_neighbors`.
            - Если количество новых соседей больше или равно минимальному требованию `min_samples`,
                то эти новые соседи добавляются в список `neighbors`.

        В конце каждой итерации цикла инкрементируется счетчик `i` для перехода к следующему
        соседу в списке `neighbors`.

        """
        self.labels[point_index] = cluster_label  # Присваиваем метку кластера выбранной точке

        i = 0
        while i < len(neighbors):  # Пока не исчерпаны все соседи
            neighbor = neighbors[i]  # Берем очередного соседа

            if self.labels[neighbor] == -1:  # Если сосед не имеет метки кластера
                self.labels[neighbor] = cluster_label  # Присваиваем ему метку кластера
            elif self.labels[neighbor] == 0:  # Если сосед не принадлежит ни одному кластеру
                self.labels[neighbor] = cluster_label  # Присваиваем ему метку кластера
                new_neighbors = self._find_neighbors(X, neighbor)  # Находим новых соседей для данной точки

                if len(new_neighbors) >= self.min_samples:  # Если количество новых соседей больше или равно минимальному требованию
                    neighbors += new_neighbors  # Добавляем новых соседей в список соседей
            i += 1  # Переходим к следующему соседу


    def _find_neighbors(self, X, point_index):
        # Инициализируем пустой список для хранения индексов соседних точек
        neighbors = []
        for i in range(len(X)):  # Проходим по всем точкам в датасете
            # Если расстояние между текущей точкой и точкой с индексом i меньше или равно eps
            if self.euclidean_distance(X[point_index], X[i]) <= self.eps:
                neighbors.append(i)  # Добавляем индекс соседней точки в список neighbors
        return neighbors  # Возвращаем список соседних точек


In [ ]:
# Пример использования
X = np.array([[1, 2], [1.5, 1.8], [5, 8], [8, 8], [1, 0.6], [9, 11]])
eps = 2
min_samples = 2

dbscan = DBSCAN(eps, min_samples)
dbscan.fit(X)

print("Метки кластеров:")
print(dbscan.labels)

Благодаря способности выявлять выбросы в данных, алгоритм DBSCAN удобно использовать для поиска аномалий в данных.

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt

# Генерация данных с несколькими кластерами
X, _ = make_blobs(n_samples=1500, centers=4, cluster_std=1.6, random_state=42)

# Масштабируем данные
X_scaled = StandardScaler().fit_transform(X)

Определим оптимальное значение `eps` с помощью NearestNeighbors и локтевой диаграммы.

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Найдем оптимальное значение eps с помощью графика k-distance
nearest_neighbors = NearestNeighbors(n_neighbors=5)
nearest_neighbors.fit(X_scaled)
distances, indices = nearest_neighbors.kneighbors(X_scaled)

# Сортируем расстояния для визуализации
distances = np.sort(distances[:, 4])  # Берем расстояния до 5-го соседа
plt.figure(figsize=(8, 6))
plt.plot(distances)
plt.title('График расстояний (k-distance graph)')
plt.xlabel('Точки данных (индексы)')
plt.ylabel('Расстояние до 5-го ближайшего соседа')
plt.grid(True)
plt.show()


In [ ]:
# Применяем DBSCAN с подобранным eps
dbscan = DBSCAN(eps=0.17, min_samples=25)  # Используйте значение eps, которое подходит для ваших данных
y_dbscan = dbscan.fit_predict(X_scaled)

# Визуализация результатов
plt.figure(figsize=(10, 7))
unique_labels = set(y_dbscan)

for label in unique_labels:
    if label == -1:  # Шумовые точки
        color = 'red'
        label_name = 'Шум'
    else:
        color = plt.cm.viridis(label / max(unique_labels))
        label_name = f'Кластер {label}'

    plt.scatter(X_scaled[y_dbscan == label, 0], X_scaled[y_dbscan == label, 1],
                c=color, label=label_name, s=10, alpha=0.7)

plt.title('DBSCAN на сгенерированных данных (4 кластера)')
plt.xlabel('Признак 1 (норм.)')
plt.ylabel('Признак 2 (норм.)')
plt.legend(markerscale=2)
plt.grid(True)
plt.show()


<a name='3'></a>
## Агломеративная кластеризация

Агломеративная кластеризация (агломеративный метод) - это алгоритм иерархической кластеризации, который последовательно объединяет близкие точки данных в кластеры, формируя дерево иерархии кластеров, называемое дендрограммой.

Процесс работы агломеративной кластеризации следующий:

1. Инициализация: Каждая точка данных начинает в качестве отдельного кластера.
2. Вычисление матрицы расстояний: Вычисляется матрица расстояний между каждой парой кластеров. Расстояние может быть определено различными способами, такими как евклидово расстояние или корреляция.
3. Объединение ближайших кластеров: Два ближайших кластера (т.е. с наименьшим расстоянием между ними) объединяются в один новый кластер.
4. Обновление матрицы расстояний: Матрица расстояний обновляется, чтобы отразить расстояния между новым объединенным кластером и остальными кластерами.
5. Повторение шагов 3 и 4: Шаги 3 и 4 повторяются до тех пор, пока все точки данных не объединятся в один кластер или достигнется заранее заданное количество кластеров.
6. Построение дендрограммы: На основе последовательности объединений строится дендрограмма, которая визуализирует иерархию кластеров.

Результат агломеративной кластеризации представляет собой дерево иерархии кластеров, где каждый узел представляет собой кластер, а расстояние между узлами отображает степень их схожести. По анализу дендрограммы можно определить оптимальное количество кластеров, а также их структуру и иерархические отношения. Агломеративная кластеризация особенно полезна в задачах, где требуется иерархическое представление кластеров или когда количество кластеров неизвестно заранее.

In [ ]:
class AgglomerativeClustering:
    def __init__(self, n_clusters):
        self.n_clusters = n_clusters
        self.labels = None

    def fit(self, X):
        # Инициализация каждой точки как отдельного кластера
        clusters = [[x] for x in X]

        while len(clusters) > self.n_clusters:  # Пока количество кластеров больше заданного числа
            min_dist = np.inf  # Инициализация минимального расстояния как бесконечность
            merge_indices = (0, 0)  # Инициализация индексов кластеров для объединения

            # Находим два ближайших кластера для объединения
            for i in range(len(clusters)):  # Проходим по каждому кластеру
                for j in range(i+1, len(clusters)):  # Проходим по остальным кластерам
                    dist = self._distance(clusters[i], clusters[j])  # Вычисляем расстояние между кластерами

                    if dist < min_dist:  # Если текущее расстояние меньше минимального
                        min_dist = dist  # Обновляем значение минимального расстояния
                        merge_indices = (i, j)  # Обновляем индексы кластеров для объединения

            # Объединяем два ближайших кластера
            merged_cluster = clusters[merge_indices[0]] + clusters[merge_indices[1]]

            # Удаляем объединенные кластеры из списка
            del clusters[merge_indices[1]]
            del clusters[merge_indices[0]]

            # Добавляем объединенный кластер в список
            clusters.append(merged_cluster)

        # Присваиваем метки кластеров точкам
        self.labels = self._assign_labels(clusters, X)


    def _distance(self, cluster1, cluster2):
        min_dist = np.inf  # Инициализируем переменную минимального расстояния как бесконечность

        for point1 in cluster1:  # Проходим по каждой точке в первом кластере
            for point2 in cluster2:  # Проходим по каждой точке во втором кластере
                dist = np.linalg.norm(point1 - point2)  # Вычисляем Евклидово расстояние между двумя точками
                if dist < min_dist:  # Если текущее расстояние меньше минимального
                    min_dist = dist  # Обновляем значение минимального расстояния

        return min_dist  # Возвращаем минимальное расстояние


    def _assign_labels(self, clusters, dataset):
        labels = np.zeros(dataset.shape[0], dtype=int)  # Заполняем для каждой точки массив меток нулями

        for index, point in enumerate(dataset):  # Проходим по каждой  точке в датасете
            for cluster_index, cluster in enumerate(clusters): # Проходим по всем кластерам
                if np.sum(np.all(cluster == point, axis=1)): # Проверяем вхождение точки в текущий кластер
                    labels[index] = cluster_index  # Присваиваем метку кластера текущей точке
                    continue

        return labels  # Возвращаем массив меток


In [ ]:
# Пример использования
X = np.array([[1, 2], [1.5, 1.8], [5, 8], [8, 8], [1, 0.6], [9, 11]])
n_clusters = 2

agglomerative = AgglomerativeClustering(n_clusters)
agglomerative.fit(X)

print("Метки кластеров:")
print(agglomerative.labels)

### Методы объединения кластеров

- Single linkage (ссылочное соединение) — расстояние между ближайшими точками двух кластеров.
- Complete linkage (полное соединение) — расстояние между самыми удалёнными точками двух кластеров.
- Average linkage (среднее соединение) — среднее расстояние между всеми точками двух кластеров.
- Ward linkage — минимизация суммы квадратов отклонений внутри кластеров, что приводит к более компактным кластерам.

В примере ниже мы используем метод `single`, который приводит к "цепочечному" объединению кластеров. Попробуйте различные методы, чтобы увидеть их влияние на структуру кластеров.

### Размерность данных
При использовании агломеративной кластеризации важно учитывать размерность данных. На высокоразмерных данных дендрограмма может быть трудной для интерпретации, и рекомендуется использовать методы снижения размерности, такие как PCA (главные компоненты), для визуализации.

### Точки на границе кластеров
Агломеративная кластеризация не всегда может дать чёткие результаты на данных, где кластеры сильно перекрываются. Это важно учитывать, когда вы выбираете метрики расстояния и методы агломерации.

### Число кластеров `n_clusters`
В агломеративной кластеризации количество кластеров определяется заранее. В случае, если вы хотите визуализировать результат в виде дендрограммы, оптимальное количество кластеров можно определить, проведя анализ на дендрограмме, например, с использованием "обрезки" дерева на определённом уровне.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.datasets import make_blobs

# Генерация случайных данных для кластеризации
X, _ = make_blobs(n_samples=20, centers=3, random_state=42)

# Применение агломеративной кластеризации с использованием scipy
linked = linkage(X, 'single')  # Использование метода 'single' для вычисления расстояний

# Построение дендрограммы
plt.figure(figsize=(8, 6))
dendrogram(linked)
plt.title("Дендрограмма для агломеративной кластеризации")
plt.xlabel("Индекс объекта")
plt.ylabel("Расстояние")
plt.show()



Для примера работы агрометративной кластеризации, давайте рассмотрим как она справляется с кластеризацией на уже знакомом нам наборе данных Iris Petals. Для удобства визуализации, мы возьмём только два признака.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA

# Загрузка набора данных Ирисы Фишера
iris = load_iris()
X = iris.data[:, 1:]  # Два признака данных
y = iris.target  # Истинные метки классов (для сравнения)

# Применение агломеративной кластеризации
n_clusters = 3  # У нас три типа ирисов, поэтому количество кластеров = 3
agg_clustering = AgglomerativeClustering(n_clusters=n_clusters)
labels = agg_clustering.fit_predict(X)


# Визуализация результатов кластеризации
plt.figure(figsize=(8, 6))
plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis', s=50)
plt.title("Агломеративная кластеризация на данных о цветках ириса")
plt.xlabel("PC1 (Главная компонента 1)")
plt.ylabel("PC2 (Главная компонента 2)")
plt.colorbar(label='Кластер')
plt.show()

# Для сравнения с истинными метками:
plt.figure(figsize=(8, 6))
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='viridis', s=50)
plt.title("Истинные метки классов на данных о цветках ириса")
plt.xlabel("PC1 (Главная компонента 1)")
plt.ylabel("PC2 (Главная компонента 2)")
plt.colorbar(label='Метка класса')
plt.show()
